# Vraag 2 - Opdracht 2
 Bouw een systeem waar je anomalies detecteert aan de hand van: autoencoder.

## Importeer benodigde bibliotheken

In [ ]:
# Importeer de benodigde bibliotheken
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

## Laad en bescrhijf de dataset

In [ ]:
# Laad de dataset
# Voeg uw bestandspad naar 'creditcard.csv' toe
data = pd.read_csv('creditcard.csv')
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


Controleer op nullen

In [ ]:
data.isnull().sum().sum()

0

In [ ]:
data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

De enige niet vervormde variablen zijn:
- Time
- Amount
- Class (1: fraude & 0: niet fraude)

In [ ]:
vc = data['Class'].value_counts().to_frame().reset_index()
vc['percent'] = vc["Class"].apply(lambda x : round(100*float(x) / len(data), 2))
vc = vc.rename(columns = {"index" : "Target", "Class" : "Count"})
vc

,Target,Count,percent
0,0,284315,99.83
1,1,492,0.17


## Data Voorbereiden

Data Preprocess

In [ ]:
# Balanceren van de dataset door niet-frauduleuze transacties te samplen
non_fraud = data[data['Class'] == 0].sample(1000)
fraud = data[data['Class'] == 1]
df = non_fraud.append(fraud).sample(frac=1).reset_index(drop=True)

<ipython-input-6-d7edfade2f42>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = non_fraud.append(fraud).sample(frac=1).reset_index(drop=True)


Het gebruik van slechts 1000 niet-frauduleuze transacties in de dataset is een strategie om het probleem van de sterk onevenwichtige verdeling aan te pakken. In de oorspronkelijke dataset zijn frauduleuze transacties (klasse 1) slechts 0,17% van het totaal, wat betekent dat er een enorme oververtegenwoordiging is van niet-frauduleuze (klasse 0) transacties. Door het aantal niet-frauduleuze gevallen te beperken tot 1000, verminder je dit onevenwicht aanzienlijk. Dit maakt het model minder bevooroordeeld tegenover het meerderheidsklasse en helpt het om betere, meer gebalanceerde inzichten te leren uit de gegevens. Dit is vooral belangrijk in representatie-leertechnieken, zoals het gebruik van auto-encoders, waarbij een evenwichtigere dataset kan leiden tot een betere algemene prestatie van het model.

Data Splitsen

In [ ]:
# Splitsen van kenmerken en labels
X = df.drop(['Class'], axis=1).values
Y = df["Class"].values

## Autoencoder model


Model Builden

In [ ]:
# Autoencoder architectuur
# Bouwen van een autoencoder voor dimensionaliteitsreductie en kenmerkextractie

# input layer
input_layer = Input(shape=(X.shape[1],))

# encoding layer
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

# decoding layer
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

# output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

Gegevens *schalen*

In [ ]:
# Schalen van de kenmerken voor betere modelprestaties
x = data.drop(["Class"], axis=1)
y = data["Class"].values

# MinMaxScaler voor de training
x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]

Model trainen

In [ ]:
# Train de autoencoder
# De autoencoder wordt getraind op niet-frauduleuze transacties
autoencoder.fit(x_norm[0:2000], x_norm[0:2000], batch_size=256, epochs=10, shuffle=True, validation_split=0.20)

Epoch 1/10
7/7 [==============================] - 3s 36ms/step - loss: 0.2680 - val_loss: 0.2668
Epoch 2/10
7/7 [==============================] - 0s 14ms/step - loss: 0.2677 - val_loss: 0.2665
Epoch 3/10
7/7 [==============================] - 0s 13ms/step - loss: 0.2674 - val_loss: 0.2662
Epoch 4/10
7/7 [==============================] - 0s 14ms/step - loss: 0.2671 - val_loss: 0.2659
Epoch 5/10
7/7 [==============================] - 0s 13ms/step - loss: 0.2668 - val_loss: 0.2656
Epoch 6/10
7/7 [==============================] - 0s 12ms/step - loss: 0.2665 - val_loss: 0.2653
Epoch 7/10
7/7 [==============================] - 0s 13ms/step - loss: 0.2662 - val_loss: 0.2650
Epoch 8/10
7/7 [==============================] - 0s 13ms/step - loss: 0.2659 - val_loss: 0.2647
Epoch 9/10
7/7 [==============================] - 0s 12ms/step - loss: 0.2656 - val_loss: 0.2644
Epoch 10/10
7/7 [==============================] - 0s 12ms/step - loss: 0.2653 - val_loss: 0.2641


 2000 rows van niet fraude cases uitleg :


 **Toelichting**: De keuze voor kleine steekproeven uit de oorspronkelijke dataset is gebaseerd op de intuïtie dat de kenmerken van de ene klasse (niet-fraude) zullen verschillen van die van de andere (fraude). Om deze kenmerken te onderscheiden, moeten we de autoencoders slechts één gegevensklasse laten zien. Dit komt omdat de autoencoder zal proberen slechts één klasse te leren en automatisch de andere klasse zal onderscheiden.

Extractie van laatste representaties van de input geleerd door het model

In [ ]:
# Extractie van latente representaties
# Gebruik van de getrainde autoencoder (eerste 3 lagen, tem encoding) om kenmerken te extraheren
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

# Genereer de verborgen representaties van twee klassen:
# niet-fraude en fraude door de ruwe invoer te voorspellen met behulp van het bovenstaande model
norm_hid_rep = hidden_representation.predict(x_norm[:3000])
fraud_hid_rep = hidden_representation.predict(x_fraud)


16/16 [==============================] - 0s 2ms/step


Voorbereiding van de uiteindelijke dataset voor classificatie

In [ ]:
# Samenvoegen van de kenmerken van beide klassen voor de uiteindelijke dataset
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis=0) # mergen tot 1 array
y_n = np.zeros(norm_hid_rep.shape[0]) # alle 0 waarden == x-aantal niet frauduleuze transacties
y_f = np.ones(fraud_hid_rep.shape[0]) # alle 1 waarden == x-aantal frauduleuze transacties
rep_y = np.append(y_n, y_f) # labels mergen tot 1 array overeenkomend met de transacties van rep_x


Simpele Lineaire Classifier

In [ ]:
# Splitsen van de dataset in trainings- en validatiesets
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)

# Trainen van een logistisch regressiemodel voor classificatie
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)


Evaluatie

In [ ]:
# Evaluatie
# Functie om evaluatiemetrieken af te drukken
def print_test_score(label, prediction):
    print("Test Resultaat:\n================================================")
    print(f"Nauwkeurigheids Score: {accuracy_score(label, prediction) * 100:.2f}%")
    print("_______________________________________________")
    print(f"Classificatie Rapport:\n{classification_report(label, prediction)}")
    print("_______________________________________________")
    print(f"Confusion Matrix: \n {confusion_matrix(label, prediction)}\n")

# Evalueer het model op de testset
print_test_score(val_y, pred_y)

# ware/verkeerde positieven/negatieven
(tn, fp,
 fn, tp) = confusion_matrix(val_y, pred_y).flatten()

# Berekenen van precisie en recall
print(f"""
% van transacties gelabeld als fraude die correct waren (precisie): {tp}/({fp}+{tp}) = {tp/(fp+tp):.2%}
% van succesvol gevangen frauduleuze transacties (recall):    {tp}/({fn}+{tp}) = {tp/(fn+tp):.2%}
""")

Test Resultaat:
Nauwkeurigheids Score: 97.14%
_______________________________________________
Classificatie Rapport:
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       742
         1.0       1.00      0.81      0.89       131

    accuracy                           0.97       873
   macro avg       0.98      0.90      0.94       873
weighted avg       0.97      0.97      0.97       873

_______________________________________________
Confusion Matrix: 
 [[742   0]
 [ 25 106]]


% van transacties gelabeld als fraude die correct waren (precisie): 106/(0+106) = 100.00%
% van succesvol gevangen frauduleuze transacties (recall):    106/(25+106) = 80.92%



---

## Confusion Matrix reports:
`[[TN, FP] `

**True Negatives (TN):** Dit zijn de gevallen waarin uw model correct voorspelt dat een transactie normaal is.

**False Positives (FP):** Dit zijn de gevallen waarin uw model onterecht voorspelt dat een normale transactie frauduleus is.


`[FN, TP]]`

**False Negatives (FN):** Dit zijn de frauduleuze transacties die uw model niet heeft gedetecteerd.


**True Positives (TP):** Dit zijn de gevallen waarin uw model correct voorspelt dat de transactie frauduleus is.

---

## Classification Report:
Het classificatierapport bevat verschillende belangrijke statistieken:

**Precision**: Geeft aan hoeveel van de door het model als positief (frauduleus) geclassificeerde gevallen daadwerkelijk positief zijn.

**Recall**: Geeft aan hoeveel van de werkelijke positieve gevallen het model correct heeft geïdentificeerd.

**F1-score**: De harmonische gemiddelde van precisie en recall, een balans tussen de twee.

**Support**: Het aantal werkelijke gevallen voor elke klasse in de dataset.

**Accuracy**: Het totale percentage correcte voorspellingen door het model.

**Macro avg**: Het ongewogen gemiddelde van de statistieken voor elke klasse.

**Weighted avg**: Het gewogen gemiddelde van de statistieken voor elke klasse, waarbij de gewichten de ondersteuning (aantal gevallen) voor elke klasse zijn.




